In [9]:
import requests
from bs4 import BeautifulSoup
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from collections import Counter
from torch.cuda.amp import GradScaler, autocast
import time
import random


In [10]:
# Fetching Orhun inscription data from multiple URLs
urls = [
    "https://tr.wikisource.org/wiki/Orhun_Yazıtları_(Bilge_Kağan)",
    "https://tr.wikisource.org/wiki/Orhun_Yazıtları_(Kül_Tigin)",
    "https://tr.wikisource.org/wiki/Orhun_Yazıtları_(Tonyukuk)"
]

def fetch_text_data(urls):
    text_data = []
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        tables = soup.find_all("table")
        for table in tables:
            rows = table.find_all("tr")
            for row in rows:
                cells = row.find_all(["td", "th"])
                if len(cells) > 2:
                    cell_text = cells[2].get_text(strip=True)
                    if cell_text and not any(keyword in cell_text for keyword in ["Başlık", "Türkçe"]):
                        text_data.append(cell_text)
    return " ".join(text_data)

text = fetch_text_data(urls)


In [11]:
# Cleaning text to remove unnecessary characters
def clean_text(text):
    text = re.sub(r'[0-9a-zA-Z,().[\]\n ]', '', text)
    return text

clean_text_data = clean_text(text)
print(clean_text_data[:1000])  # İlk 1000 karakteri yazdır


ğşıüğığ𐱅𐰭𐰼𐰃:𐱅𐰏:𐱅𐰭𐰼𐰃:𐰖𐰺𐱃𐰢𐱁:𐱅𐰇𐰼𐰰:𐰋𐰃𐰠𐰏𐰀:𐰴𐰍𐰣:𐰽𐰉𐰢:𐰴𐰭𐰢:𐱅𐰇𐰼𐰰:𐰋𐰃𐰠𐰏𐰀:𐰦𐰀:𐰞𐱃𐰃:𐰾𐰃𐰼:𐱃𐰆𐰴𐰕:𐰆𐰍𐰕:𐰘𐰃:𐰓𐰕:𐰚𐰼𐰚𐰇𐰠𐰏:𐰋𐰏𐰠𐰼𐰃:𐰉𐰆𐰑𐰣𐰃::𐱅𐰇𐰼𐰰:𐱅𐰭𐰼𐰃:𐰇𐰕𐰀:𐰴𐰍𐰣:𐰆𐰞𐰺𐱃𐰢:𐰆𐰞𐰺𐱃𐰸𐰢𐰀:𐰇𐰠𐱅𐰲𐰃𐰲𐰀:𐰽𐰴𐰣𐰍𐰢𐰀:𐱅𐰇𐰼𐰰:𐰋𐰏𐰠𐰼:𐰉𐰆𐰑𐰣:𐰏𐰼𐰯:𐰾𐰋𐰤𐰯:𐱃𐰆𐰭𐱃𐰢𐱁:𐰚𐰇𐰕𐰃:𐰘𐰇𐰏𐰼𐰚𐰇𐰼𐱅𐰃:𐰋𐰇𐰓𐰚𐰀:𐰇𐰕𐰢:𐰆𐰞𐰺𐰯:𐰉𐰆𐰨𐰀:𐰍𐰺:𐱅𐰇𐰼𐰏:𐱅𐰇𐰼𐱅:𐰉𐰆𐰞𐰭𐰑𐰴𐰃::𐰃𐱅𐰓𐰢:𐰇𐰕𐰀:𐰚𐰇𐰚:𐱅𐰭𐰼𐰃:𐰽𐰺𐰀:𐰖𐰍𐰕:𐰘𐰺:𐰴𐰃𐰞𐰦𐰸𐰑𐰀:𐰚𐰃𐰤:𐰺𐰀:𐰚𐰃𐰾𐰃:𐰆𐰍𐰞𐰃:𐰴𐰃𐰞𐰣𐰢𐰾:𐰚𐰃𐰾𐰃:𐰆𐰍𐰞𐰃𐰦𐰀:𐰇𐰕𐰀:𐰲𐰇𐰢:𐰯𐰀𐰢:𐰉𐰆𐰢𐰣:𐰴𐰍𐰣:𐰃𐰾𐱅𐰢𐰃:𐰴𐰍𐰣:𐰆𐰞𐰺𐰢𐱁:𐰆𐰞𐰺𐰯𐰣:𐱅𐰇𐰼𐰰:𐰉𐰆𐰑𐰣𐰭:𐰃𐰠𐰃𐰤:𐱅𐰇𐰼𐰇𐰾𐰃𐰤:𐱃𐰆𐱃𐰀:𐰋𐰃𐰼𐰢𐰾:𐰃𐱅𐰃:𐰋𐰃𐰼𐰢𐰾:𐱅𐰇𐰼𐱅:𐰉𐰆𐰞𐰭:𐰴𐰆𐰯:𐰖𐰍𐰃:𐰼𐰢𐰾:𐰾𐰇:𐰾𐰇𐰠𐰯𐰤:𐱅𐰇𐰼𐱅:𐰉𐰆𐰞𐰭𐰑𐰴𐰃:𐰉𐰆𐰑𐰣𐰍:𐰴𐰆𐰯:𐰞𐰢𐰾:𐰚𐰆𐰯:𐰉𐰕:𐰴𐰃𐰞𐰢𐰾:𐰉𐱁𐰞𐰍𐰍:𐰘𐰰𐰦𐰼𐰢𐰾:𐱅𐰃𐰕𐰲𐰏𐰏:𐰾𐰇𐰚𐰇𐰼𐰢𐰾:𐰃𐰠𐰏𐰠𐰇:𐰴𐰑𐰺𐰴𐰣:𐰘𐰃𐱁𐰴𐰀:𐱅𐰏𐰃:𐰚𐰃𐰼𐰇:𐱅𐰢𐰼𐰴𐰯𐰍𐰴𐰀:𐱅𐰏𐰃:𐰸𐰆𐰦𐰺𐰢𐰾:𐰚𐰃𐰤:𐰺𐰀:𐰃𐰓𐰃:𐰆𐰴𐰽𐰕:𐰰𐰇𐰚:𐱅𐰇𐰼𐰰:𐰨𐰀:𐰆𐰞𐰺𐰆𐰺:𐰼𐰢𐰾:𐰋𐰃𐰠𐰏𐰀:𐰴𐰍𐰣:𐰼𐰢𐰾:𐰞𐰯:𐰴𐰍𐰣:𐰼𐰢𐰾:𐰉𐰆𐰖𐰺𐰸𐰃:𐰘𐰢𐰀:𐰋𐰃𐰠𐰏𐰀:𐰼𐰢𐰾:𐰼𐰨:𐰞𐰯:𐰼𐰢𐰾:𐰼𐰨:𐰋𐰏𐰠𐰼𐰃:𐰘𐰢𐰀:𐰉𐰆𐰑𐰣𐰃:𐰘𐰢𐰀:𐱅𐰇𐰕:𐰼𐰢𐰾:𐰣𐰃:𐰇𐰲𐰇𐰤:𐰃𐰠𐰏:𐰨𐰀:𐱃𐰆𐱃𐰢𐰾:𐰼𐰨:𐰃𐰠𐰏:𐱃𐰆𐱃𐰯:𐱅𐰇𐰼𐰇𐰏:𐰃𐱅𐰢𐰾:𐰇𐰕𐰃:𐰨𐰀:𐰚𐰼𐰏𐰚:𐰉𐰆𐰞𐰢𐰾:𐰖𐰆𐰍𐰲𐰃:𐰾𐰃𐰍𐱃𐰲𐰃:𐰇𐰭𐰼𐰃:𐰰𐰇𐰤:𐱃𐰆𐰍𐰽𐰴𐰑𐰃:𐰋𐰇𐰚𐰲𐰃:𐰲𐰇𐰠𐰏𐰠:𐱃𐰉𐰍𐰲:𐱅𐰇𐰯𐰇𐱅:𐰯𐰺:𐰯𐰆𐰺𐰢:𐰶𐰃𐰺𐰴𐰕:𐰇𐰲𐰸𐰆𐰺𐰃𐰴𐰣:𐰆𐱃𐰕𐱃𐱃𐰺:𐰶𐰃𐱃𐰪:𐱃𐱃𐰉𐰃:𐰉𐰆𐰨𐰀:𐰉𐰆𐰑𐰣:𐰚𐰠𐰯𐰤:𐰽𐰃𐰍𐱃𐰀𐰢𐰾:𐰖𐰆𐰍𐰞𐰀𐰢𐰾:𐰦𐰍:𐰚𐰇𐰠𐰏:𐰴𐰍𐰣:𐰼𐰢𐰾:𐰦𐰀:𐰚𐰃𐰾𐰼𐰀:𐰃𐰤𐰃𐰾𐰃:𐰴𐰍𐰣:𐰉𐰆𐰞𐰢𐰾:𐰼𐰨:𐰆𐰍𐰞𐰃𐱃𐰃:𐰴𐰍𐰣:𐰉𐰆𐰞𐰢𐰾:𐰼𐰨:𐰦𐰀:𐰚𐰃𐰾𐰼𐰀:𐰃𐰤𐰃𐰾𐰃:𐰲𐰃𐰾𐰃𐰤:𐱅𐰏:𐰴𐰃𐰞𐰣𐰢𐰑𐰸:𐰼𐰨:𐰆𐰍𐰞𐰃:𐰴𐰭𐰃𐰤:𐱅𐰏:𐰴𐰃𐰞𐰣𐰢𐰑𐰸:𐰼𐰨:𐰋𐰃𐰠𐰏𐰾𐰃𐰕:𐰴𐰍𐰣:𐰆𐰞𐰺𐰢𐰾:

In [12]:
# Computing word and character frequencies
def compute_frequencies(text):
    words = text.split()
    word_freq = Counter(words)
    char_freq = Counter(text)
    return word_freq, char_freq

word_frequencies, char_frequencies = compute_frequencies(clean_text_data)
print("Word Frequencies:", word_frequencies.most_common(10))
print("Character Frequencies:", char_frequencies.most_common(10))


Word Frequencies: [('ğşıüğığ𐱅𐰭𐰼𐰃:𐱅𐰏:𐱅𐰭𐰼𐰃:𐰖𐰺𐱃𐰢𐱁:𐱅𐰇𐰼𐰰:𐰋𐰃𐰠𐰏𐰀:𐰴𐰍𐰣:𐰽𐰉𐰢:𐰴𐰭𐰢:𐱅𐰇𐰼𐰰:𐰋𐰃𐰠𐰏𐰀:𐰦𐰀:𐰞𐱃𐰃:𐰾𐰃𐰼:𐱃𐰆𐰴𐰕:𐰆𐰍𐰕:𐰘𐰃:𐰓𐰕:𐰚𐰼𐰚𐰇𐰠𐰏:𐰋𐰏𐰠𐰼𐰃:𐰉𐰆𐰑𐰣𐰃::𐱅𐰇𐰼𐰰:𐱅𐰭𐰼𐰃:𐰇𐰕𐰀:𐰴𐰍𐰣:𐰆𐰞𐰺𐱃𐰢:𐰆𐰞𐰺𐱃𐰸𐰢𐰀:𐰇𐰠𐱅𐰲𐰃𐰲𐰀:𐰽𐰴𐰣𐰍𐰢𐰀:𐱅𐰇𐰼𐰰:𐰋𐰏𐰠𐰼:𐰉𐰆𐰑𐰣:𐰏𐰼𐰯:𐰾𐰋𐰤𐰯:𐱃𐰆𐰭𐱃𐰢𐱁:𐰚𐰇𐰕𐰃:𐰘𐰇𐰏𐰼𐰚𐰇𐰼𐱅𐰃:𐰋𐰇𐰓𐰚𐰀:𐰇𐰕𐰢:𐰆𐰞𐰺𐰯:𐰉𐰆𐰨𐰀:𐰍𐰺:𐱅𐰇𐰼𐰏:𐱅𐰇𐰼𐱅:𐰉𐰆𐰞𐰭𐰑𐰴𐰃::𐰃𐱅𐰓𐰢:𐰇𐰕𐰀:𐰚𐰇𐰚:𐱅𐰭𐰼𐰃:𐰽𐰺𐰀:𐰖𐰍𐰕:𐰘𐰺:𐰴𐰃𐰞𐰦𐰸𐰑𐰀:𐰚𐰃𐰤:𐰺𐰀:𐰚𐰃𐰾𐰃:𐰆𐰍𐰞𐰃:𐰴𐰃𐰞𐰣𐰢𐰾:𐰚𐰃𐰾𐰃:𐰆𐰍𐰞𐰃𐰦𐰀:𐰇𐰕𐰀:𐰲𐰇𐰢:𐰯𐰀𐰢:𐰉𐰆𐰢𐰣:𐰴𐰍𐰣:𐰃𐰾𐱅𐰢𐰃:𐰴𐰍𐰣:𐰆𐰞𐰺𐰢𐱁:𐰆𐰞𐰺𐰯𐰣:𐱅𐰇𐰼𐰰:𐰉𐰆𐰑𐰣𐰭:𐰃𐰠𐰃𐰤:𐱅𐰇𐰼𐰇𐰾𐰃𐰤:𐱃𐰆𐱃𐰀:𐰋𐰃𐰼𐰢𐰾:𐰃𐱅𐰃:𐰋𐰃𐰼𐰢𐰾:𐱅𐰇𐰼𐱅:𐰉𐰆𐰞𐰭:𐰴𐰆𐰯:𐰖𐰍𐰃:𐰼𐰢𐰾:𐰾𐰇:𐰾𐰇𐰠𐰯𐰤:𐱅𐰇𐰼𐱅:𐰉𐰆𐰞𐰭𐰑𐰴𐰃:𐰉𐰆𐰑𐰣𐰍:𐰴𐰆𐰯:𐰞𐰢𐰾:𐰚𐰆𐰯:𐰉𐰕:𐰴𐰃𐰞𐰢𐰾:𐰉𐱁𐰞𐰍𐰍:𐰘𐰰𐰦𐰼𐰢𐰾:𐱅𐰃𐰕𐰲𐰏𐰏:𐰾𐰇𐰚𐰇𐰼𐰢𐰾:𐰃𐰠𐰏𐰠𐰇:𐰴𐰑𐰺𐰴𐰣:𐰘𐰃𐱁𐰴𐰀:𐱅𐰏𐰃:𐰚𐰃𐰼𐰇:𐱅𐰢𐰼𐰴𐰯𐰍𐰴𐰀:𐱅𐰏𐰃:𐰸𐰆𐰦𐰺𐰢𐰾:𐰚𐰃𐰤:𐰺𐰀:𐰃𐰓𐰃:𐰆𐰴𐰽𐰕:𐰰𐰇𐰚:𐱅𐰇𐰼𐰰:𐰨𐰀:𐰆𐰞𐰺𐰆𐰺:𐰼𐰢𐰾:𐰋𐰃𐰠𐰏𐰀:𐰴𐰍𐰣:𐰼𐰢𐰾:𐰞𐰯:𐰴𐰍𐰣:𐰼𐰢𐰾:𐰉𐰆𐰖𐰺𐰸𐰃:𐰘𐰢𐰀:𐰋𐰃𐰠𐰏𐰀:𐰼𐰢𐰾:𐰼𐰨:𐰞𐰯:𐰼𐰢𐰾:𐰼𐰨:𐰋𐰏𐰠𐰼𐰃:𐰘𐰢𐰀:𐰉𐰆𐰑𐰣𐰃:𐰘𐰢𐰀:𐱅𐰇𐰕:𐰼𐰢𐰾:𐰣𐰃:𐰇𐰲𐰇𐰤:𐰃𐰠𐰏:𐰨𐰀:𐱃𐰆𐱃𐰢𐰾:𐰼𐰨:𐰃𐰠𐰏:𐱃𐰆𐱃𐰯:𐱅𐰇𐰼𐰇𐰏:𐰃𐱅𐰢𐰾:𐰇𐰕𐰃:𐰨𐰀:𐰚𐰼𐰏𐰚:𐰉𐰆𐰞𐰢𐰾:𐰖𐰆𐰍𐰲𐰃:𐰾𐰃𐰍𐱃𐰲𐰃:𐰇𐰭𐰼𐰃:𐰰𐰇𐰤:𐱃𐰆𐰍𐰽𐰴𐰑𐰃:𐰋𐰇𐰚𐰲𐰃:𐰲𐰇𐰠𐰏𐰠:𐱃𐰉𐰍𐰲:𐱅𐰇𐰯𐰇𐱅:𐰯𐰺:𐰯𐰆𐰺𐰢:𐰶𐰃𐰺𐰴𐰕:𐰇𐰲𐰸𐰆𐰺𐰃𐰴𐰣:𐰆𐱃𐰕𐱃𐱃𐰺:𐰶𐰃𐱃𐰪:𐱃𐱃𐰉𐰃:𐰉𐰆𐰨𐰀:𐰉𐰆𐰑𐰣:𐰚𐰠𐰯𐰤:𐰽𐰃𐰍𐱃𐰀𐰢𐰾:𐰖𐰆𐰍𐰞𐰀𐰢𐰾:𐰦𐰍:𐰚𐰇𐰠𐰏:𐰴𐰍𐰣:𐰼𐰢𐰾:𐰦𐰀:𐰚𐰃𐰾𐰼𐰀:𐰃𐰤𐰃𐰾𐰃:𐰴𐰍𐰣:𐰉𐰆𐰞𐰢𐰾:𐰼𐰨:𐰆𐰍𐰞𐰃𐱃𐰃:𐰴𐰍𐰣:𐰉𐰆𐰞𐰢𐰾:𐰼𐰨:𐰦𐰀:𐰚𐰃𐰾𐰼𐰀:𐰃𐰤𐰃𐰾𐰃:𐰲𐰃𐰾𐰃𐰤:𐱅𐰏:𐰴𐰃𐰞𐰣𐰢𐰑𐰸:𐰼𐰨:𐰆𐰍𐰞𐰃:𐰴𐰭𐰃𐰤:𐱅𐰏:𐰴𐰃𐰞𐰣𐰢𐰑𐰸:

In [13]:
# Create mappings from ancient to modern alphabet
def create_alphabet_mappings(text):
    ancient_alphabet = ''.join(set(text))
    modern_alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    
    if len(ancient_alphabet) > len(modern_alphabet):
        repeats = len(ancient_alphabet) // len(modern_alphabet) + 1
        modern_alphabet = (modern_alphabet * repeats)[:len(ancient_alphabet)]
    
    return ancient_alphabet, modern_alphabet

ancient_alphabet, modern_alphabet = create_alphabet_mappings(clean_text_data)
print("Ancient Alphabet:", ancient_alphabet)
print("Modern Alphabet:", modern_alphabet)


Ancient Alphabet: 𐰯𐱁𐰠𐰝ı𐰕𐱅𐰤𐰓𐰀𐰶𐰚𐱈𐰭𐰍𐰣𐱃𐰏𐰰𐰱𐰺𐰾ü𐰋𐰸𐰇𐰑 𐰦ş𐰨𐰆𐰽:𐰪𐰉𐰴𐰘𐰼ğ𐰞𐰡𐰲𐰃𐰢𐰖
Modern Alphabet: ABCDEFGHIJKLMNOPQRSTUVWXYZABCDEFGHIJKLMNOPQRST


In [14]:
# Preparing data by aligning characters between ancient and modern alphabets
def prepare_data_with_frequencies(text, ancient_alphabet, modern_alphabet, word_frequencies, char_frequencies, direction='ltr'):
    if direction == 'rtl':
        text = text[::-1]
    pairs = []
    words = text.split()
    for word in words:
        if word:
            ancient_indices = [ancient_alphabet.index(c) for c in word if c in ancient_alphabet]
            modern_indices = [modern_alphabet.index(random.choice(modern_alphabet)) for _ in ancient_indices]
            pairs.extend([(a, m) for a, m in zip(ancient_indices, modern_indices)])
    return pairs

# Asking for reading direction
direction = input("Choose reading direction (ltr/rtl): ").strip().lower()
pairs = prepare_data_with_frequencies(clean_text_data, ancient_alphabet, modern_alphabet, word_frequencies, char_frequencies, direction)
print("Number of pairs:", len(pairs))


Number of pairs: 28695


In [15]:
# Define dataset class for the mappings
class AlphabetDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x, y = self.data[idx]
        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)

train_pairs, test_pairs = train_test_split(pairs, test_size=0.2, random_state=42)
train_dataset = AlphabetDataset(train_pairs)
test_dataset = AlphabetDataset(test_pairs)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)


In [16]:
# Define Transformer-based model
class TransformerDecoderModel(nn.Module):
    def __init__(self, input_dim, output_dim, emb_dim, n_layers, n_heads, pf_dim, dropout):
        super(TransformerDecoderModel, self).__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.transformer = nn.Transformer(
            d_model=emb_dim, 
            nhead=n_heads, 
            num_encoder_layers=n_layers, 
            num_decoder_layers=n_layers, 
            dim_feedforward=pf_dim,
            dropout=dropout
        )
        self.fc_out = nn.Linear(emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, trg):
        src = self.embedding(src)
        trg = self.embedding(trg)

        src = src.permute(1, 0, 2)
        trg = trg.permute(1, 0, 2)

        output = self.transformer(src, trg)
        output = output.permute(1, 0, 2)

        return self.fc_out(output)

# Model hyperparameters
input_dim = len(ancient_alphabet)
output_dim = len(modern_alphabet)
emb_dim = 512
n_layers = 6
n_heads = 8
pf_dim = 1024
dropout = 0.1

# Create model instance on CPU
model = TransformerDecoderModel(input_dim, output_dim, emb_dim, n_layers, n_heads, pf_dim, dropout)

In [17]:
# Mixed precision scaler
scaler = GradScaler()

# Training function with mixed precision and optimization
def train_model(model, dataloader, criterion, optimizer, scheduler, epochs=20):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        start_time = time.time()
        for batch in tqdm(dataloader, desc=f'Epoch {epoch + 1}/{epochs}', unit='batch'):
            x, y = batch
            x, y = x.cuda(), y.cuda()
            
            optimizer.zero_grad()
            
            # Mixed precision
            with autocast():
                output = model(x, y)
                loss = criterion(output.view(-1, output_dim), y.view(-1))
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            epoch_loss += loss.item()
        
        scheduler.step()
        end_time = time.time()
        print(f'Epoch {epoch + 1}, Loss: {epoch_loss / len(dataloader)}, Time: {end_time - start_time:.2f}s')

# Define loss and optimizer with learning rate scheduling
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

# Train the model with mixed precision and optimizations
train_model(model, train_loader, criterion, optimizer, scheduler, epochs=20)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11908\2484566989.py:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Epoch 1/20:   0%|          | 0/359 [00:00<?, ?batch/s]

In [ ]:
# Add evaluation function with similar optimizations
def evaluate_model(model, dataloader, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            x, y = batch
            x, y = x.cuda(), y.cuda()

            with autocast():
                output = model(x, y)
                loss = criterion(output.view(-1, output_dim), y.view(-1))

            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

# Validate the model
val_loss = evaluate_model(model, test_loader, criterion)
print(f'Validation Loss: {val_loss}')

# Function to evaluate and map predictions
def evaluate_model_predictions(model, test_data):
    model.eval()
    predictions = []
    with torch.no_grad():
        for (x, y) in test_data:
            x_tensor = torch.tensor(x, dtype=torch.long).unsqueeze(0).cuda()
            output = model(x_tensor, x_tensor)
            pred_idx = output.argmax(dim=-1).squeeze().tolist()
            pred_text = ''.join([modern_alphabet[i] for i in pred_idx])
            predictions.append(pred_text)
    return predictions

predicted_text = evaluate_model_predictions(model, test_pairs)
print(predicted_text[:10])  # İlk 10 tahmini yazdır


In [ ]:
# Searching for culturally and historically relevant words
def search_relevant_words(decoded_text, relevant_words):
    found_words = {word: [] for word in relevant_words}
    for word in relevant_words:
        start = 0
        while True:
            start = decoded_text.find(word, start)
            if start == -1:
                break
            found_words[word].append(start)
            start += len(word)
    return found_words

# Relevant words list
relevant_words = ['tanrı', 'ilah', 'yüce', 'kudretli', 'türk', 'türük', 'türki', 'göktürk',
                  'kağan', 'han', 'devlet', 'ordu', 'er', 'alp', 'yiğit', 'ata', 'baba', 'soy', 'ulus']

found_relevant_words = search_relevant_words(predicted_text, relevant_words)

print("\nFound Relevant Words:")
for word, positions in found_relevant_words.items():
    if positions:
        print(f"{word}: Found at positions {positions}")
    else:
        print(f"{word}: Not found")
